# LiquidGenerator

In [1]:
from ase.build import molecule
from ase.io import write

ethanol = molecule("CH3CH2OH")
ethanol.set_cell([10.0, 10.0, 10.0]) 
ethanol.set_pbc([True, True, True]) 

write("CH3CH2OH.vasp", ethanol)

In [2]:
from ase import Atoms
from ase.build import molecule
from ase.data import atomic_numbers, atomic_masses
from ase.io import read, write
import numpy as np

class LiquidGenerator:
    def __init__(self, target_density, cell_dimensions=(10.0, 10.0, 10.0), grid_size=5.0, skin=0.25):
        """
        初始化液体生成器。
        Args:
            target_density (float): 目标密度 (g/cm³)。
            cell_dimensions (tuple): 晶胞尺寸 (a, b, c) （Å）。
            grid_size (float): 网格的边长（Å）。
            skin (float): 外层屏蔽区域厚度（Å）。
        """
        self.target_density = target_density
        self.cell_dimensions = cell_dimensions
        self.grid_size = grid_size
        self.skin = skin
        self.molecule_species = []
        self.total_mass = 0
        self.all_atoms = None

    def calculate_molecular_weight(self, atoms):
        """
        根据 ASE 的 Atoms 对象计算分子的摩尔质量。
        Args:
            atoms (Atoms): ASE 的 Atoms 对象。
        Returns:
            float: 分子的摩尔质量 (g/mol)。
        """
        symbols = atoms.get_chemical_symbols()
        return sum(atomic_masses[atomic_numbers[symbol]] for symbol in symbols)
        
    def add_molecule(self, atoms=None, name=None, count=1, min_distances=1.0, mol_weights=None):
        """
        添加一种分子到混合体系。
        Args:
            atoms (atoms): 分子。
            name (str): 分子名称（如 "H2O"）。
            count (int): 分子数量。
            min_distances (float): 分子间最小距离（Å）。
            mol_weights (float): 分子的摩尔质量 (g/mol)，如果未提供，将自动计算。
        """
        if atoms is None:
            if not name:
                raise ValueError("Either 'atoms' or 'name' must be provided.")
            atoms = molecule(name)

        if mol_weights is None:
            mol_weights = self.calculate_molecular_weight(atoms)

        self.molecule_species.append({
            "atoms": atoms,
            "name": name or "customed molecule",
            "count": count,
            "min_distances": min_distances,
            "mol_weights": mol_weights
        })
        self.total_mass += count * mol_weights / 6.02214076e23

    def calculate_volume(self):
        """
        根据目标密度和分子质量计算晶胞体积。
        Returns:
            float: 晶胞体积 (Å³)。
        """
        return self.total_mass / self.target_density * 1e24

    def adjust_cell_dimensions(self):
        """
        根据目标密度调整晶胞尺寸比例。
        """
        volume = self.calculate_volume()
        a, b, c = self.cell_dimensions
        scale = (volume / (a * b * c)) ** (1 / 3)
        self.cell_dimensions = (a * scale, b * scale, c * scale)

    def is_valid_blue_noise(self, pos, positions, min_dist):
        for existing_pos in positions:
            if np.linalg.norm(pos - existing_pos) < min_dist:
                return False
        return True

    def blue_noise_sampling(self, num_points, cell_dimensions, min_dist, adjusted_skin):
        positions = []
        a, b, c = cell_dimensions
        while len(positions) < num_points:
            pos = np.random.rand(3) * np.array([a, b, c])
            if (adjusted_skin < pos[0] < a - adjusted_skin and
                adjusted_skin < pos[1] < b - adjusted_skin and
                adjusted_skin < pos[2] < c - adjusted_skin and
                self.is_valid_blue_noise(pos, positions, min_dist)):
                positions.append(pos)
        return positions

    def generate_structure(self):
        self.adjust_cell_dimensions()
        a, b, c = self.cell_dimensions
        print(f"Adjusted cell dimensions: a={a:.2f} Å, b={b:.2f} Å, c={c:.2f} Å")

        self.all_atoms = Atoms(cell=[(a, 0, 0), (0, b, 0), (0, 0, c)], pbc=True)

        for spec in self.molecule_species:
            mol = spec["atoms"]
            mol_name = spec["name"]
            mol_count = spec["count"]
            min_dist = spec["min_distances"]

            max_extent = max(np.linalg.norm(atom.position - mol.get_center_of_mass()) for atom in mol)
            adjusted_skin = self.skin + max_extent

            positions = self.blue_noise_sampling(mol_count, self.cell_dimensions, min_dist, adjusted_skin)

            for pos in positions:
                mol_copy = mol.copy()
                mol_copy.rotate(np.random.rand() * 360, 'x')
                mol_copy.rotate(np.random.rand() * 360, 'y')
                mol_copy.rotate(np.random.rand() * 360, 'z')
                mol_copy.translate(pos)
                self.all_atoms += mol_copy

        print(f"Final density: {self.target_density} g/cm³")
        return self.all_atoms

# 使用示例
generator = LiquidGenerator(target_density=0.8, cell_dimensions=(20.0, 15.0, 10.0), grid_size=5.0, skin=0.25)
generator.add_molecule(name="H2O", count=500, min_distances=2.5)
generator.add_molecule(atoms=read("CH3CH2OH.vasp"), count=100, min_distances=3.0)
liquid_atoms = generator.generate_structure()
write("liquid.xyz", liquid_atoms)

Adjusted cell dimensions: a=42.24 Å, b=31.68 Å, c=21.12 Å
Final density: 0.8 g/cm³
